In [5]:
import geopy
import pandas as pd
import numpy as np

from pyproj import Geod
from shapely.geometry import Point, LineString

X = pd.read_csv('../data/stores_train.csv')
spatial = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
age = pd.read_csv('../data/grunnkrets_age_distribution.csv')

In [6]:
spatial.head()
age.head()
# age.loc(age.grunnkrets_id == 3011703)
age.drop_duplicates(subset='grunnkrets_id')[age.grunnkrets_id == 3011703]


/var/folders/55/fnz5gpdd2vn8_5ncwxtldqww0000gp/T/ipykernel_68270/3153172759.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  age.drop_duplicates(subset='grunnkrets_id')[age.grunnkrets_id == 3011703]


,grunnkrets_id,year,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,...,age_81,age_82,age_83,age_84,age_85,age_86,age_87,age_88,age_89,age_90
58,3011703,2015,8,8,8,8,8,7,7,6,...,1,1,1,1,1,1,1,1,1,0


In [7]:
gk_muni = spatial[['grunnkrets_id', 'municipality_name']]

old_shape = X.shape
# print(len(X.merge(gk_muni, on='grunnkrets_id', how='right')))
X = X.merge(gk_muni, on='grunnkrets_id', how='left')

_ = X.groupby(['municipality_name'])['lat', 'lon'].apply(lambda x: x.sum() / (x.count()))[['lat', 'lon']]
X = X.merge(_, on=['municipality_name'], how='left', suffixes=(None, '_center')).drop_duplicates()
assert X.shape[0] == old_shape[0]

def meter_distance(lat1, lon1, lat2, lon2):
    line_string = LineString([Point(lon1, lat1), Point(lon2, lat2)])
    geod = Geod(ellps="WGS84")
    return geod.geometry_length(line_string)


X.fillna(value={'lat_center': 63, 'lon_center': 10}, inplace=True)
X.isnull().sum()

X['dist_to_center'] = X.apply(lambda row: meter_distance(row.lat, row.lon, row.lat_center, row.lon_center), axis=1)
X[20:30]

assert X.shape[0] == old_shape[0]



/var/folders/55/fnz5gpdd2vn8_5ncwxtldqww0000gp/T/ipykernel_68270/2877985337.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  _ = X.groupby(['municipality_name'])['lat', 'lon'].apply(lambda x: x.sum() / (x.count()))[['lat', 'lon']]


In [8]:
X[['municipality_name', 'lat_center']]


,municipality_name,lat_center
0,Drammen,59.740315
2,Oslo,59.919972
4,Ringerike,60.176006
6,Bodø,67.283052
8,Trondheim,63.418063
...,...,...
25678,Horten,59.404396
25680,Gjøvik,60.806942
25682,Skedsmo,59.957614
25684,Sandnes,58.857718
